# Feathr Feature Store on Azure Demo Notebook

This notebook illustrates the use of Feature Store to create a model that predicts NYC Taxi fares. It includes these steps:


This tutorial demonstrates the key capabilities of Feathr, including:

1. Install and set up Feathr with Azure
2. Create shareable features with Feathr feature definition configs.
3. Create a training dataset via point-in-time feature join.
4. Compute and write features.
5. Train a model using these features to predict fares.
6. Materialize feature value to online store.
7. Fetch feature value in real-time from online store for online scoring.

In this tutorial, we use Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The feature flow is as below:

![Feature Flow](https://github.com/linkedin/feathr/blob/main/docs/images/feature_flow.png?raw=true)

## Prerequisite: Provision cloud resources

First step is to provision required cloud resources if you want to use Feathr. Feathr provides a python based client to interact with cloud resources.

Please follow the steps [here]() to provision required cloud resources. Due to the complexity of the possible cloud environment, it is almost impossible to create a script that works for all the use cases. Because of this, [azure_resource_provision.sh](https://github.com/linkedin/feathr/blob/main/docs/how-to-guides/azure_resource_provision.sh) is a full end to end command line to create all the required resources, and you can tailor the script as needed, while [the companion documentation](https://github.com/linkedin/feathr/blob/main/docs/how-to-guides/azure-deployment.md) can be used as a complete guide for using that shell script.

At the end of the script, it should give you some output which you will need later. For example, the Service Principal IDs, Redis endpoint, etc.

Please also note that at the end of this step, you need to **manually** grant your service principal "Data Curator" permission of your Azure Purview account, due to a current limiation with Azure Purview.

And the architecture is as below:

![Architecture](https://github.com/linkedin/feathr/blob/main/docs/images/architecture.png?raw=true)

## Prerequisite: Install Feathr

Install Feathr using pip:

`pip install -U feathr pandavro scikit-learn`

Or if you want to use the latest Feathr code from GitHub:

`pip install -I git+https://github.com/linkedin/feathr.git#subdirectory=feathr_project pandavro scikit-learn`

## Prerequisite: Configure the required environment

In the first step (Provision cloud resources), you should have provisioned all the required cloud resources. If you use Feathr CLI to create a workspace, you should have a folder with a file called `feathr_config.yaml` in it with all the required configurations. Otherwise, update the configuration below.

The code below will write this configuration string to a temporary location and load it to Feathr. Please still refer to [feathr_config.yaml](https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) and use that as the source of truth. It should also have more explanations on the meaning of each variable.

In [1]:
import tempfile
yaml_config = """
# Please refer to https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml for explanations on the meaning of each field.
api_version: 1
project_config:
  project_name: 'feathr_sample'
  required_environment_variables:
    - 'REDIS_PASSWORD'
    - 'AZURE_CLIENT_ID'
    - 'AZURE_TENANT_ID'
    - 'AZURE_CLIENT_SECRET'
offline_store:
  adls:
    adls_enabled: true
  wasb:
    wasb_enabled: true
  s3:
    s3_enabled: false
    s3_endpoint: 's3.amazonaws.com'
  jdbc:
    jdbc_enabled: false
    jdbc_database: 'feathrtestdb'
    jdbc_table: 'feathrtesttable'
  snowflake:
    url: "dqllago-ol19457.snowflakecomputing.com"
    user: "feathrintegration"
    role: "ACCOUNTADMIN"
spark_config:
  spark_cluster: 'azure_synapse'
  spark_result_output_parts: '1'
  azure_synapse:
    dev_url: 'https://feathrazuretest3synapse.dev.azuresynapse.net'
    pool_name: 'spark3'
    workspace_dir: 'abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started'
    executor_size: 'Small'
    executor_num: 4
    feathr_runtime_location: wasbs://public@azurefeathrstorage.blob.core.windows.net/feathr-assembly-LATEST.jar
  databricks:
    workspace_instance_url: 'https://adb-6885802458123232.12.azuredatabricks.net/'
    workspace_token_value: ''
    config_template: {'run_name':'','new_cluster':{'spark_version':'9.1.x-scala2.12','node_type_id':'Standard_D3_v2','num_workers':2,'spark_conf':{}},'libraries':[{'jar':''}],'spark_jar_task':{'main_class_name':'','parameters':['']}}
    work_dir: 'dbfs:/feathr_getting_started'
    feathr_runtime_location: wasbs://public@azurefeathrstorage.blob.core.windows.net/feathr-assembly-LATEST.jar
online_store:
  redis:
    host: 'feathrazuretest3redis.redis.cache.windows.net'
    port: 6380
    ssl_enabled: True
feature_registry:
  purview:
    type_system_initialization: true
    purview_name: 'feathrazuretest3-purview1'
    delimiter: '__'
"""
tmp = tempfile.NamedTemporaryFile(mode='w', delete=False)
with open(tmp.name, "w") as text_file:
    text_file.write(yaml_config)


## View the data

In this tutorial, we use Feathr Feature Store to create a model that predicts NYC Taxi fares. The dataset comes from [here](https://www1.nyc.gov/site/tlc/about/tlc-trip-record-data.page). The data is as below

In [2]:
import pandas as pd
pd.read_csv("https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2020-04.csv")


/var/folders/c0/h7cgkq4x56s__301z9203fw0001p3_/T/ipykernel_92744/3211500075.py:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  pd.read_csv("https://s3.amazonaws.com/nyc-tlc/trip+data/green_tripdata_2020-04.csv")


,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,extra,mta_tax,tip_amount,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge
0,2.0,2020-04-01 00:44:02,2020-04-01 00:52:23,N,1.0,42,41,1.0,1.68,8.00,0.5,0.5,0.0,0.00,NaN,0.3,9.30,1.0,1.0,0.0
1,2.0,2020-04-01 00:24:39,2020-04-01 00:33:06,N,1.0,244,247,2.0,1.94,9.00,0.5,0.5,0.0,0.00,NaN,0.3,10.30,2.0,1.0,0.0
2,2.0,2020-04-01 00:45:06,2020-04-01 00:51:13,N,1.0,244,243,3.0,1.00,6.50,0.5,0.5,0.0,0.00,NaN,0.3,7.80,2.0,1.0,0.0
3,2.0,2020-04-01 00:45:06,2020-04-01 01:04:39,N,1.0,244,243,2.0,2.81,12.00,0.5,0.5,0.0,0.00,NaN,0.3,13.30,2.0,1.0,0.0
4,2.0,2020-04-01 00:00:23,2020-04-01 00:16:13,N,1.0,75,169,1.0,6.79,21.00,0.5,0.5,0.0,0.00,NaN,0.3,22.30,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35607,NaN,2020-04-30 23:29:00,2020-04-30 23:57:00,NaN,NaN,37,147,NaN,11.41,35.82,0.0,0.0,0.0,6.12,NaN,0.3,42.24,NaN,NaN,NaN
35608,NaN,2020-04-30 23:11:00,2020-04-30 23:47:00,NaN,NaN,188,230,NaN,11.17,35.45,0.0,0.0,0.0,0.00,NaN,0.3,38.50,NaN,NaN,NaN
35609,NaN,2020-04-30 23:18:00,2020-04-30 23:46:00,NaN,NaN,205,37,NaN,14.37,34.54,0.0,0.0,0.0,0.00,NaN,0.3,34.84,NaN,NaN,NaN
35610,NaN,2020-04-30 23:55:00,2020-05-01 00:10:00,NaN,NaN,37,188,NaN,4.25,16.72,0.0,0.0,0.0,0.00,NaN,0.3,17.02,NaN,NaN,NaN


In [3]:
import glob
import os
import tempfile
from datetime import datetime, timedelta
from math import sqrt

import pandas as pd
import pandavro as pdx
from feathr import FeathrClient
from feathr.anchor import FeatureAnchor
from feathr.client import FeathrClient
from feathr.dtype import BOOLEAN, FLOAT, INT32, ValueType
from feathr.feature import Feature
from feathr.feature_derivations import DerivedFeature
from feathr.materialization_settings import (BackfillTime,
                                             MaterializationSettings)
from feathr.query_feature_list import FeatureQuery
from feathr.settings import ObservationSettings
from feathr.sink import RedisSink
from feathr.source import INPUT_CONTEXT, HdfsSource
from feathr.transformation import WindowAggTransformation
from feathr.typed_key import TypedKey
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split


## Setup necessary environment variables

You have to setup the environment variables in order to run this sample. More environment variables can be set by referring to [feathr_config.yaml](https://github.com/linkedin/feathr/blob/main/feathr_project/feathrcli/data/feathr_user_workspace/feathr_config.yaml) and use that as the source of truth. It should also have more explanations on the meaning of each variable.

In [4]:
os.environ['REDIS_PASSWORD'] = ''
os.environ['AZURE_CLIENT_ID'] = ''
os.environ['AZURE_TENANT_ID'] = ''
os.environ['AZURE_CLIENT_SECRET'] = ''

Then we will initialize a feathr client:


In [6]:
client = FeathrClient(config_path=tmp.name)

## Defining Features with Feathr:

In Feathr, a feature is viewed as a function, mapping from entity id or key, and timestamp to a feature value.

1. The entity key (a.k.a. entity id) identifies the subject of feature, e.g. a user id, 123.
2. The feature name is the aspect of the entity that the feature is indicating, e.g. the age of the user.
3. The feature value is the actual value of that aspect at a particular time, e.g. the value is 30 at year 2022.

Note that, in some cases, such as features defined on top of request data, may have no entity key or timestamp.
It is merely a function/transformation executing against request data at runtime.
For example, the day of week of the request, which is calculated by converting the request UNIX timestamp.

```python
batch_source = HdfsSource(name="nycTaxiBatchSource",
                          path="abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/green_tripdata_2020-04.csv",
                          event_timestamp_column="lpep_dropoff_datetime",
                          timestamp_format="yyyy-MM-dd HH:mm:ss")


f_trip_distance = Feature(name="f_trip_distance",
                          feature_type=FLOAT, transform="trip_distance")
f_trip_time_duration = Feature(name="f_trip_time_duration",
                               feature_type=INT32,
                               transform="time_duration(lpep_pickup_datetime, lpep_dropoff_datetime, 'minutes')")

features = [
    f_trip_distance,
    f_trip_time_duration,
    Feature(name="f_is_long_trip_distance",
            feature_type=BOOLEAN,
            transform="cast_float(trip_distance)>30"),
    Feature(name="f_day_of_week",
            feature_type=INT32,
            transform="dayofweek(lpep_dropoff_datetime)"),
]

request_anchor = FeatureAnchor(name="request_features",
                               source=INPUT_CONTEXT,
                               features=features)
```

In [7]:
batch_source = HdfsSource(name="nycTaxiBatchSource",
                          path="abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/green_tripdata_2020-04.csv",
                          event_timestamp_column="lpep_dropoff_datetime",
                          timestamp_format="yyyy-MM-dd HH:mm:ss")


f_trip_distance = Feature(name="f_trip_distance",
                          feature_type=FLOAT, transform="trip_distance")
f_trip_time_duration = Feature(name="f_trip_time_duration",
                               feature_type=INT32,
                               transform="time_duration(lpep_pickup_datetime, lpep_dropoff_datetime, 'minutes')")

features = [
    f_trip_distance,
    f_trip_time_duration,
    Feature(name="f_is_long_trip_distance",
            feature_type=BOOLEAN,
            transform="cast_float(trip_distance)>30"),
    Feature(name="f_day_of_week",
            feature_type=INT32,
            transform="dayofweek(lpep_dropoff_datetime)"),
]

request_anchor = FeatureAnchor(name="request_features",
                               source=INPUT_CONTEXT,
                               features=features)


f_trip_time_distance = DerivedFeature(name="f_trip_time_distance",
                                      feature_type=FLOAT,
                                      input_features=[
                                          f_trip_distance, f_trip_time_duration],
                                      transform="f_trip_distance * f_trip_time_duration")

f_trip_time_rounded = DerivedFeature(name="f_trip_time_rounded",
                                     feature_type=INT32,
                                     input_features=[f_trip_time_duration],
                                     transform="f_trip_time_duration % 10")


location_id = TypedKey(key_column="DOLocationID",
                       key_column_type=ValueType.INT32,
                       description="location id in NYC",
                       full_name="nyc_taxi.location_id")
agg_features = [Feature(name="f_location_avg_fare",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(fare_amount)",
                                                          agg_func="AVG",
                                                          window="90d")),
                Feature(name="f_location_max_fare",
                        key=location_id,
                        feature_type=FLOAT,
                        transform=WindowAggTransformation(agg_expr="cast_float(fare_amount)",
                                                          agg_func="MAX",
                                                          window="90d"))
                ]

agg_anchor = FeatureAnchor(name="aggregationFeatures",
                           source=batch_source,
                           features=agg_features)

And then we need to build those features so that it can be consumed later. Note that we have to build both the "anchor" and the "derived" features (which is not anchored to a source).

In [8]:
client.build_features(anchor_list=[agg_anchor, request_anchor], derived_feature_list=[
                      f_trip_time_distance, f_trip_time_rounded])

## Create training data using point-in-time correct feature join

A training dataset usually contains entity id columns, multiple feature columns, event timestamp column and label/target column. 

To create a training dataset using Feathr, one needs to provide a feature join configuration file to specify
what features and how these features should be joined to the observation data. The feature join config file mainly contains: 

1. The path of a dataset as the 'spine' for the to-be-created training dataset. We call this input 'spine' dataset the 'observation'
   dataset. Typically, each row of the observation data contains: 
   a) Column(s) representing entity id(s), which will be used as the join key to look up(join) feature value. 
   b) A column representing the event time of the row. By default, Feathr will make sure the feature values joined have
   a timestamp earlier than it, ensuring no data leakage in the resulting training dataset. 
   c) Other columns will be simply pass through onto the output training dataset.
2. The key fields from the observation data, which are used to joined with the feature data.
3. List of feature names to be joined with the observation data. The features must be defined in the feature
   definition configs.
4. The time information of the observation data used to compare with the feature's timestamp during the join.

Create training dataset via:

```python
client.get_offline_features(observation_settings=settings,
                            feature_query=feature_query,
                            output_path="abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/output.avro")
```


In [9]:
feature_query = FeatureQuery(
    feature_list=["f_location_avg_fare", "f_trip_time_rounded", "f_is_long_trip_distance"], key=location_id)
settings = ObservationSettings(
    observation_path="abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/green_tripdata_2020-04.csv",
    event_timestamp_column="lpep_dropoff_datetime",
    timestamp_format="yyyy-MM-dd HH:mm:ss")
client.get_offline_features(observation_settings=settings,
                            feature_query=feature_query,
                            output_path="abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/output.avro")
client.wait_job_to_finish(timeout_sec=500)

2022-03-21 21:01:37.964 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:38 - Uploading /var/folders/c0/h7cgkq4x56s__301z9203fw0001p3_/T/tmp6v6yjpzw/feature_join_conf/feature_join.conf to cloud..
2022-03-21 21:01:37.965 | INFO     | feathr._synapse_submission:upload_file:317 - Uploading file feature_join.conf
2022-03-21 21:01:38.418 | INFO     | feathr._synapse_submission:upload_file:323 - /var/folders/c0/h7cgkq4x56s__301z9203fw0001p3_/T/tmp6v6yjpzw/feature_join_conf/feature_join.conf is uploaded to location: abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/feature_join.conf
2022-03-21 21:01:38.419 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:41 - /var/folders/c0/h7cgkq4x56s__301z9203fw0001p3_/T/tmp6v6yjpzw/feature_join_conf/feature_join.conf is uploaded to location: abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/feature_join.conf
2022-03-21 21:01:38.419 | I

## Download the result and show the result

Let's use the helper function `get_result_df` to download the result and view it:

In [10]:
def get_result_df(client: FeathrClient) -> pd.DataFrame:
    """Download the job result dataset from cloud as a Pandas dataframe."""
    res_url = client.get_job_result_uri(block=True, timeout_sec=600)
    tmp_dir = tempfile.TemporaryDirectory()
    client.feathr_spark_laucher.download_result(result_path=res_url, local_folder=tmp_dir.name)
    dataframe_list = []
    # assuming the result are in avro format
    for file in glob.glob(os.path.join(tmp_dir.name, '*.avro')):
        dataframe_list.append(pdx.read_avro(file))
    vertical_concat_df = pd.concat(dataframe_list, axis=0)
    tmp_dir.cleanup()
    return vertical_concat_df

df_res = get_result_df(client)

2022-03-21 21:03:42.075 | INFO     | feathr._synapse_submission:wait_for_completion:109 - Current Spark job status: success
2022-03-21 21:03:42.174 | INFO     | feathr._synapse_submission:download_file:334 - Beginning reading of results from abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/output.avro
2022-03-21 21:03:58.599 | INFO     | feathr._synapse_submission:download_file:355 - Finish downloading files from abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/demo_data/output.avro to /var/folders/c0/h7cgkq4x56s__301z9203fw0001p3_/T/tmpn9_zja46.


In [11]:
df_res

,VendorID,lpep_pickup_datetime,lpep_dropoff_datetime,store_and_fwd_flag,RatecodeID,PULocationID,DOLocationID,passenger_count,trip_distance,fare_amount,...,tolls_amount,ehail_fee,improvement_surcharge,total_amount,payment_type,trip_type,congestion_surcharge,f_is_long_trip_distance,f_location_avg_fare,f_trip_time_rounded
0,2,2020-04-01 09:04:53,2020-04-01 09:16:06,N,5,42,147,1,3.36,15,...,0,None,0.3,16.18,1,2,0,False,15.0,1
1,2,2020-04-01 10:59:12,2020-04-01 11:14:49,N,1,75,147,1,2.50,13,...,0,None,0.3,13.8,2,1,0,False,15.0,5
2,2,2020-04-01 15:01:09,2020-04-01 15:14:03,N,1,42,147,1,2.48,11.5,...,0,None,0.3,12.3,2,1,0,False,15.0,2
3,2,2020-04-01 15:34:25,2020-04-01 15:48:27,N,1,42,147,1,3.80,14.5,...,0,None,0.3,15.3,2,1,0,False,15.0,4
4,2,2020-04-01 19:37:30,2020-04-01 19:50:10,N,1,42,147,1,2.95,12,...,0,None,0.3,13.8,2,1,0,False,15.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
507,None,2020-04-30 08:38:00,2020-04-30 08:47:00,None,None,247,247,None,1.61,7.86,...,0,None,0.3,9.79,None,None,None,False,9.0,9
508,None,2020-04-30 10:36:00,2020-04-30 10:52:00,None,None,238,247,None,7.34,16.25,...,0,None,0.3,19.3,None,None,None,False,9.0,6
509,None,2020-04-30 11:27:00,2020-04-30 11:35:00,None,None,42,247,None,1.44,7.1,...,0,None,0.3,8.88,None,None,None,False,9.0,8
510,None,2020-04-30 12:10:00,2020-04-30 12:16:00,None,None,235,247,None,.97,8,...,0,None,0.3,11.05,None,None,None,False,9.0,6


## Train a machine learning model
After getting all the features, let's train a machine learning model with the converted feature by Feathr:

In [12]:
# remove columns
from sklearn.ensemble import GradientBoostingRegressor
final_df = df_res
final_df.drop(["lpep_pickup_datetime", "lpep_dropoff_datetime",
              "store_and_fwd_flag"], axis=1, inplace=True, errors='ignore')
final_df.fillna(0, inplace=True)
final_df['fare_amount'] = final_df['fare_amount'].astype("float64")


train_x, test_x, train_y, test_y = train_test_split(final_df.drop(["fare_amount"], axis=1),
                                                    final_df["fare_amount"],
                                                    test_size=0.2,
                                                    random_state=42)
model = GradientBoostingRegressor()
model.fit(train_x, train_y)

y_predict = model.predict(test_x)

y_actual = test_y.values.flatten().tolist()
rmse = sqrt(mean_squared_error(y_actual, y_predict))

sum_actuals = sum_errors = 0

for actual_val, predict_val in zip(y_actual, y_predict):
    abs_error = actual_val - predict_val
    if abs_error < 0:
        abs_error = abs_error * -1

    sum_errors = sum_errors + abs_error
    sum_actuals = sum_actuals + actual_val

mean_abs_percent_error = sum_errors / sum_actuals
print("Model MAPE:")
print(mean_abs_percent_error)
print()
print("Model Accuracy:")
print(1 - mean_abs_percent_error)


Model MAPE:
0.02498496557554859

Model Accuracy:
0.9750150344244514


## Materialize feature value into offline/online storage

While Feathr can compute the feature value from the feature definition on-the-fly at request time, it can also pre-compute
and materialize the feature value to offline and/or online storage. 

We now want to push the generated features to the online store, so we configure the destination in the feature_gen config:


In [13]:
backfill_time = BackfillTime(start=datetime(
    2020, 5, 20), end=datetime(2020, 5, 20), step=timedelta(days=1))
redisSink = RedisSink(table_name="nycTaxiDemoFeature")
settings = MaterializationSettings("nycTaxiTable",
                                   backfill_time=backfill_time,
                                   sinks=[redisSink],
                                   feature_names=["f_location_avg_fare", "f_location_max_fare"])

client.materialize_features(settings)
client.wait_job_to_finish(timeout_sec=500)


2022-03-21 21:04:02.559 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:38 - Uploading /var/folders/c0/h7cgkq4x56s__301z9203fw0001p3_/T/tmp6v6yjpzw/feature_gen_conf/auto_gen_config_1589958000.0.conf to cloud..
2022-03-21 21:04:02.560 | INFO     | feathr._synapse_submission:upload_file:317 - Uploading file auto_gen_config_1589958000.0.conf
2022-03-21 21:04:03.092 | INFO     | feathr._synapse_submission:upload_file:323 - /var/folders/c0/h7cgkq4x56s__301z9203fw0001p3_/T/tmp6v6yjpzw/feature_gen_conf/auto_gen_config_1589958000.0.conf is uploaded to location: abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.windows.net/feathr_getting_started/auto_gen_config_1589958000.0.conf
2022-03-21 21:04:03.094 | INFO     | feathr._synapse_submission:upload_or_get_cloud_path:41 - /var/folders/c0/h7cgkq4x56s__301z9203fw0001p3_/T/tmp6v6yjpzw/feature_gen_conf/auto_gen_config_1589958000.0.conf is uploaded to location: abfss://feathrazuretest3fs@feathrazuretest3storage.dfs.core.win

We can then get the features from the online store (Redis):


## Fetching feature value for online inference

For features that are already materialized by the previous step, their latest value can be queried via the client's
`get_online_features` or `multi_get_online_features` API.

```python
client.get_online_features("nycTaxiDemoFeature", "265", ['f_location_avg_fare', 'f_location_max_fare'])
client.multi_get_online_features("nycTaxiDemoFeature", ["239", "265"], ['f_location_avg_fare', 'f_location_max_fare'])
```


In [14]:
res = client.get_online_features('nycTaxiDemoFeature', '265', [
                                 'f_location_avg_fare', 'f_location_max_fare'])

In [15]:
client.multi_get_online_features("nycTaxiDemoFeature", ["239", "265"], [
                                 'f_location_avg_fare', 'f_location_max_fare'])


{'239': [10.5, 10.5], '265': [42.5, 42.5]}

In [16]:
client.list_registered_features(project_name="frame_getting_started")

['f_location_max_fare',
 'f_location_avg_fare',
 'f_trip_distance',
 'f_trip_time_duration',
 'f_hour_of_day',
 'f_is_long_trip_distance',
 'f_day_of_month',
 'f_day_of_week']